In [ ]:
# Object Detection with TensorFlow (Kaggle Dataset)

import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# Kaggle setup for downloading dataset
!pip install kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"your_kaggle_username","key":"your_kaggle_api_key"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Download the Face Mask Detection Dataset from Kaggle
!kaggle datasets download -d andrewmvd/face-mask-detection
!unzip face-mask-detection.zip

# Set up paths
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'models'
ANNOTATION_PATH = WORKSPACE_PATH + '/annotations'
IMAGE_PATH = 'images'
MODEL_PATH = WORKSPACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/ssd_mobilenet_v2_fpnlite/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/ssd_mobilenet_v2_fpnlite'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-0')).expect_partial()

# Load label map data
category_index = {1: {'id': 1, 'name': 'with_mask'}, 2: {'id': 2, 'name': 'without_mask'}}

# Detection function
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Test with a sample image
import cv2
import numpy as np
from matplotlib import pyplot as plt

IMAGE_PATH = 'images/maksssksksss0.png'
img = cv2.imread(IMAGE_PATH)
input_tensor = tf.convert_to_tensor(np.expand_dims(img, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

viz_utils.visualize_boxes_and_labels_on_image_array(
    img,
    detections['detection_boxes'][0].numpy(),
    (detections['detection_classes'][0].numpy() + 1).astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=0.5,
    agnostic_mode=False)

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

print("Object Detection Project Completed!")
